In [3]:
import pandas as pd
import os
import numpy as np
from scipy.stats import skew
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

(a)

In [4]:
# 读取“MarketValue”内所有csv文件
market = pd.DataFrame()
notebook_directory = os.path.dirname(os.path.abspath("__file__"))
daily_directory = os.path.join(notebook_directory, 'MarketValue')

for root_dir, sub_dir, files in os.walk(daily_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root_dir, file)
            df = pd.read_csv(file_path)
            df.rename(columns = {'Symbol' : 'Code'}, inplace = True)
            df['Month'] = df['TradingDate'].astype(str).str[:7]
            market = pd.concat([market, df], ignore_index=True)

In [5]:
# 读取“DailyReturn”内所有csv文件
returns = pd.DataFrame()
notebook_directory = os.path.dirname(os.path.abspath("__file__"))
daily_directory = os.path.join(notebook_directory, 'DailyReturn')

for root_dir, sub_dir, files in os.walk(daily_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root_dir, file)
            df = pd.read_csv(file_path)
            df.rename(columns = {'Stkcd' : 'Code', 'Trddt': 'TradingDate'}, inplace = True)
            returns = pd.concat([returns, df], ignore_index=True)

In [6]:
# 合并dataframe，并计算EP ratio
df_merged = pd.merge(market, returns, on=['TradingDate', 'Code'])
df_merged['EP'] = df_merged['PE'].apply(lambda x: 1/x)
df_merged = df_merged.drop(['PE'], axis=1)
df_merged.head()

,TradingDate,Code,ShortName,CirculatedMarketValue,Month,Dretwd,EP
0,2000-01-04,600000,浦发银行,8.182400e+09,2000-01,0.033131,0.015038
1,2000-01-05,600000,浦发银行,8.089600e+09,2000-01,-0.011341,0.015210
2,2000-01-06,600000,浦发银行,8.316800e+09,2000-01,0.028085,0.014795
3,2000-01-07,600000,浦发银行,8.608000e+09,2000-01,0.035013,0.014294
4,2000-01-10,600000,浦发银行,8.720000e+09,2000-01,0.013011,0.014111


In [7]:
# 获取每个月月末日期
market['TradingDate'] = pd.to_datetime(market['TradingDate'])
df_dates = market.groupby('Month')['TradingDate'].last()
dates_list = df_dates.tolist()

In [8]:
# 筛选出所有月需要剔除掉的股票代码，并保留前70%的股票代码
result_symbols = []

for date in dates_list:
    df_date = market[market['TradingDate'] == date]
    df_date = df_date.sort_values(by='CirculatedMarketValue', ascending=False)
    num_off = int(len(df_date) * 0.3)   # 计算后30%的股票数量
    symbols = df_date.iloc[-num_off:]['Code'].tolist()   # 得到后30%股票代码
    filtered_codes = df_date.loc[~df_date['Code'].isin(symbols), 'Code'].tolist()  # 保留剔除掉后30%的股票代码

    result_symbols.append(filtered_codes)

In [22]:
# 计算MKT因子

# 读取文件
df_rf = pd.read_csv('TRD_Nrrate.csv')
df_market_monthly = pd.read_csv('TRD_Mont.csv')

df_rf['Nrrmtdt'] = df_rf['Nrrmtdt'] / 100  # 把百分比转换为小数

# 按月份分组，并选择每个组的第一个日期对应的Nrrmtdt，存储月度无风险收益到rf_factor中
df_rf['Clsdt'] = pd.to_datetime(df_rf['Clsdt'])
rf_factor = df_rf.groupby(df_rf['Clsdt'].dt.to_period("M"))['Nrrmtdt'].first().tolist()

# 存储市场收益到rm_factor中
df_market_monthly = df_market_monthly[df_market_monthly['Markettype'] == 1]  # 上证A股市场
rm_factor = df_market_monthly['Mretwdos'].tolist()

# 计算MKT因子，rm - rf
MKT_factors_list = np.subtract(rm_factor, rf_factor)


In [42]:
# 计算SMB和HML因子和Rt
size_factors_list = []
value_factors_list = []
Rt_list = []
prvs_SMB = 1
prvs_HML = 1

months_list = df_merged['Month'].unique().tolist()

for index, month in enumerate(months_list):
    symbols_for_month = result_symbols[index]  # 获得当月的股票代码
    df_month = df_merged[(df_merged['Month'] == month) & df_merged['Code'].isin(symbols_for_month)]  # 获得当月市值前70%的股票数据

    # 求Rt
    monthly_return_list_1 = []
    df_daily_stock_rt = df_month.groupby('Code')

    for _, df in df_daily_stock_rt:
        indiv_return = (1 + df['Dretwd']).prod() - 1     # 对于每只股票，计算月收益率
        monthly_return_list_1.append(indiv_return)

    Rt = np.mean(monthly_return_list_1)
    Rt_list.append(Rt)
    
    # 求SMB
    monthly_return_list = []
    df_month_mv = df_month.sort_values(by='CirculatedMarketValue', ascending=False)  # 按照CirculatedMarketValue排序
    df_daily_stock_mv = df_month_mv.groupby('Code')

    for _, df in df_daily_stock_mv:
        indiv_return = (1 + df['Dretwd']).prod() - 1     # 对于每只股票，计算月收益率
        monthly_return_list.append(indiv_return)

    num_bs = int(len(symbols_for_month) * 0.5)
    small_mean = np.mean(monthly_return_list[-num_bs:])  # 市值后50%的股票月收益率
    big_mean = np.mean(monthly_return_list[:num_bs])     # 市值前50%的股票月收益率

    if prvs_SMB < 0:                                     # 若上月SMB<0, 则本月改变策略，用big - small作为size factor
        size_factor = big_mean - small_mean
    else:
        size_factor = small_mean - big_mean                  
    size_factors_list.append(size_factor)

    # 求HML
    monthly_return_list_2 = []
    df_month_ep = df_month.sort_values(by='EP', ascending=False)  # 按照EP ratio排序
    df_daily_stock_ep = df_month_ep.groupby('Code')

    for _, df in df_daily_stock_ep:
        indiv_return_2 = (1 + df['Dretwd']).prod() - 1   # 对于每只股票，计算月收益率
        monthly_return_list_2.append(indiv_return_2)
    
    num_hl = int(len(symbols_for_month) * 0.3)
    high_mean = np.mean(monthly_return_list_2[:num_hl], dtype=np.float64)  # EP前30%的股票月收益率
    low_mean = np.mean(monthly_return_list_2[-num_hl:], dtype=np.float64)  # EP后30%的股票月收益率

    if prvs_HML < 0:                                     # 若上月HML<0, 则本月改变策略，用low - high作为value factor
        value_factor = low_mean - high_mean
    else:
        value_factor = high_mean - low_mean
    value_factors_list.append(value_factor)

    prvs_SMB = size_factor
    prvs_HML = value_factor

In [79]:
# 构建CH-3因子模型

X_MKT = np.array(MKT_factors_list).reshape(-1, 1)
X_SMB = np.array(size_factors_list).reshape(-1, 1)
X_VMG = np.array(value_factors_list).reshape(-1, 1)
y_Rt = np.array(Rt_list)

# 合并 X_MKT、X_SMB、X_VMG 为一个特征矩阵 X
X = np.concatenate((X_MKT, X_SMB, X_VMG), axis=1)

# 创建并拟合 LinearRegression 模型
model = LinearRegression()
model.fit(X, y_Rt)

# 输出模型的截距和系数
print("Intercept (alpha):", model.intercept_)
print("Coefficients (beta_MKT, beta_SMB, beta_VMG):", model.coef_)

Intercept (alpha): 0.015434115405039318
Coefficients (beta_MKT, beta_SMB, beta_VMG): [0.58989729 0.19715176 0.33632188]


In [12]:
# 读取其他市场年化回报率文件
benchmark = pd.read_csv('TRD_Yearm.csv')
benchmark_list = benchmark[benchmark['Markettype'] == 4]['Yretwdeq'].tolist()  # 深证A股市场2000-2016年化回报率

In [80]:
# 计算每年的年化收益率、年化波动率、偏度和胜率

# 将数据按照每年12个月进行分组
groups_size = [size_factors_list[i:i+12] for i in range(0, len(size_factors_list), 12)]
groups_value = [value_factors_list[i:i+12] for i in range(0, len(value_factors_list), 12)]
benchmark_value = [benchmark_list[i] for i in range(0, len(benchmark_list))]

results_size = []
results_value = []
return_size_list = []
return_value_list = []

# 遍历每年的规模因子、价值因子、深证A股年化收益数据
for group_size, group_value, benchmark_value in zip(groups_size, groups_value, benchmark_value):
    
    # 计算年化收益率
    monthly_returns_s = np.array(group_size)
    monthly_returns_v = np.array(group_value)

    annualized_return_s = np.prod(1 + monthly_returns_s) - 1
    annualized_return_v = np.prod(1 + monthly_returns_v) - 1

    return_size_list.append(annualized_return_s)
    return_value_list.append(annualized_return_v)

    # 计算年化收益率
    annualized_volatility_size = np.std(group_size) * np.sqrt(12)
    annualized_volatility_value = np.std(group_value) * np.sqrt(12)

    # 计算每年的偏度
    skewness_size = skew(group_size)
    skewness_value = skew(group_value)

    # 将结果存储到list中
    results_size.append({
        'Annualized_Return': annualized_return_s,
        'Annualized_Volatility': annualized_volatility_size,
        'Skewness': skewness_size,
    })

    results_value.append({
        'Annualized_Return': annualized_return_v,
        'Annualized_Volatility': annualized_volatility_value,
        'Skewness': skewness_value,
    })    

return_size_array = np.array(return_size_list)
return_value_array = np.array(return_value_list)
benchmark_value_array = np.array(benchmark_value)

# 计算17年的胜率
win_rate_size = np.mean(return_size_array > benchmark_value_array)
win_rate_value = np.mean(return_value_array > benchmark_value_array)

df_results_size = pd.DataFrame(results_size) 
df_results_value = pd.DataFrame(results_value)  

df_results_size.to_excel("results_size.xlsx", index=False)      # 保存size factor的表现年化年化收益率、年化波动率、偏度在results_size.xlsx
df_results_value.to_excel("results_value.xlsx", index=False)    # 保存value factor的表现年化年化收益率、年化波动率、偏度在results_value.xlsx

print('规模因子17年胜率:', win_rate_size)
print('价值因子17年胜率:', win_rate_value)


规模因子17年胜率: 0.8235294117647058
价值因子17年胜率: 1.0


(b)

In [49]:
# 构建Market cap anomaly和EP anomaly
mc_anomaly = size_factors_list
ep_anomaly = value_factors_list

In [78]:
# 构建CAPM模型: Rit = αi + βi*(Rmt-Rft)
X = np.array(rm_factor).reshape(-1, 1)

y_mc = mc_anomaly
X_with_intercept_mc = sm.add_constant(X)
nw_mc_t = sm.OLS(y_mc, X_with_intercept_mc).fit(cov_type='HAC', cov_kwds={'maxlags': 2})

print('Market cap anomaly:')
print("α:", nw_mc_t.params[0])
print("t(α):", nw_mc_t.tvalues[0])
print("β:", nw_mc_t.params[1])
print("t(β):", nw_mc_t.tvalues[1])

y_ep = ep_anomaly
X_with_intercept_ep = sm.add_constant(X)
nw_ep_t = sm.OLS(y_ep, X_with_intercept_ep).fit(cov_type='HAC', cov_kwds={'maxlags': 2})

print('\n')
print('EP anomaly:')
print("α:", nw_ep_t.params[0])
print("t(α):", nw_ep_t.tvalues[0])
print("β:", nw_ep_t.params[1])
print("t(β):", nw_ep_t.tvalues[1])


Market cap anomaly:
α: -0.00022226178334036606
t(α): -0.19764604758571727
β: 0.00598335709154681
t(β): 0.50793252457598


EP anomaly:
α: 0.0013551508721185229
t(α): 1.3576432551479696
β: 0.0158333143839181
t(β): 1.2446622755752703


In [77]:
# 构建CH-3模型

X_MKT = np.array(MKT_factors_list).reshape(-1, 1)
X_SMB = np.array(size_factors_list).reshape(-1, 1)
X_VMG = np.array(value_factors_list).reshape(-1, 1)
X = np.concatenate((X_MKT, X_SMB, X_VMG), axis=1)

# 用Newey-west调整方差方法做OLS回归
y_mc = mc_anomaly
X_with_intercept_mc = sm.add_constant(X)
nw_mc_t = sm.OLS(y_mc, X_with_intercept_mc).fit(cov_type='HAC', cov_kwds={'maxlags': 2})

print('Market cap anomaly:')
print("α:", nw_mc_t.params[0])
print("t(α):", nw_mc_t.tvalues[0])
print("βMKT:", nw_mc_t.params[1])
print("t(βMKT):", nw_mc_t.tvalues[1])
print("βSMB:", nw_mc_t.params[2])
print("t(βSMB):", nw_mc_t.tvalues[2])
print("βHML:", nw_mc_t.params[3])
print("t(βHML):", nw_mc_t.tvalues[3])

y_ep = ep_anomaly
X_with_intercept_ep = sm.add_constant(X)
nw_ep_t = sm.OLS(y_ep, X_with_intercept_ep).fit(cov_type='HAC', cov_kwds={'maxlags': 2})

print('\n')
print('EP anomaly:')
print("α:", nw_ep_t.params[0])
print("t(α):", nw_ep_t.tvalues[0])
print("βMKT:", nw_ep_t.params[1])
print("t(βMKT):", nw_ep_t.tvalues[1])
print("βSMB:", nw_ep_t.params[2])
print("t(βSMB):", nw_ep_t.tvalues[2])
print("βHML:", nw_ep_t.params[3])
print("t(βHML):", nw_ep_t.tvalues[3])

Market cap anomaly:
α: -6.505213034913027e-19
t(α): -2.092663687693117
βMKT: 2.0816681711721685e-17
t(βMKT): 3.582280056417916
βSMB: 1.0
t(βSMB): 4.354127531212873e+16
βHML: 2.636779683484747e-16
t(βHML): 5.012284725397858


EP anomaly:
α: 1.6263032587282567e-19
t(α): 0.5022175907581705
βMKT: 2.7321894746634712e-17
t(βMKT): 4.419388057422116
βSMB: 1.0408340855860843e-16
t(βSMB): 2.611002854018177
βHML: 1.0000000000000002
t(βHML): 2.360612132522125e+16


(c)

In [55]:
# 读取CSMAR上的三因子模型月度数据
df_ff = pd.read_csv('STK_MKT_THRFACMONTH.csv')
df_ff = df_ff[df_ff['MarkettypeID'] == 'P9701']  # 上证A股

In [56]:
# 构建correlation

factors_custom = np.array([MKT_factors_list, size_factors_list, value_factors_list]).T

# 提取 CSMAR 中的三因子数据
ff_factors = df_ff[['RiskPremium1', 'SMB1', 'HML1']]
ff_factors_array = ff_factors.to_numpy()

# 计算自定义三因子模型的因子相关度矩阵
correlation_matrix_custom = np.corrcoef(factors_custom, rowvar=False)

# 计算 CSMAR 中的三因子模型的因子相关度矩阵
correlation_matrix_csmar = np.corrcoef(ff_factors_array, rowvar=False)

print("自定义三因子模型的因子相关度矩阵:")
print(correlation_matrix_custom)

print("\nCSMAR 三因子模型的因子相关度矩阵:")
print(correlation_matrix_csmar)

# 比较两个相关系数矩阵
correlation_difference = correlation_matrix_custom - correlation_matrix_csmar
print("\n两个相关系数矩阵的差异:")
print(correlation_difference)

自定义三因子模型的因子相关度矩阵:
[[ 1.          0.0256919   0.08806581]
 [ 0.0256919   1.         -0.00203989]
 [ 0.08806581 -0.00203989  1.        ]]

CSMAR 三因子模型的因子相关度矩阵:
[[ 1.          0.04465806 -0.01082801]
 [ 0.04465806  1.         -0.53136844]
 [-0.01082801 -0.53136844  1.        ]]

两个相关系数矩阵的差异:
[[ 1.11022302e-16 -1.89661578e-02  9.88938199e-02]
 [-1.89661578e-02  0.00000000e+00  5.29328553e-01]
 [ 9.88938199e-02  5.29328553e-01  0.00000000e+00]]


In [60]:
merged_factors = np.concatenate((factors_custom, ff_factors_array), axis=1)
correlation_matrix = np.corrcoef(merged_factors, rowvar=False)

print("\nCovariance Matrix:")
print(correlation_matrix)


Covariance Matrix:
[[ 1.          0.0256919   0.08806581  0.97776886  0.16249019 -0.06218438]
 [ 0.0256919   1.         -0.00203989  0.03575471 -0.10602126  0.11305955]
 [ 0.08806581 -0.00203989  1.          0.10506245 -0.01768804  0.09304416]
 [ 0.97776886  0.03575471  0.10506245  1.          0.04465806 -0.01082801]
 [ 0.16249019 -0.10602126 -0.01768804  0.04465806  1.         -0.53136844]
 [-0.06218438  0.11305955  0.09304416 -0.01082801 -0.53136844  1.        ]]


(d)

In [81]:
# 读取“DailyReturn d”内所有csv文件
returns = pd.DataFrame()
notebook_directory = os.path.dirname(os.path.abspath("__file__"))
daily_directory = os.path.join(notebook_directory, 'DailyReturn d')

for root_dir, sub_dir, files in os.walk(daily_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root_dir, file)
            df = pd.read_csv(file_path)
            df.rename(columns = {'Stkcd' : 'Code', 'Trddt': 'TradingDate'}, inplace = True)
            returns = pd.concat([returns, df], ignore_index=True)

In [85]:
# 读取“MarketValue d”内所有csv文件
market = pd.DataFrame()
notebook_directory = os.path.dirname(os.path.abspath("__file__"))
daily_directory = os.path.join(notebook_directory, 'MarketValue d')

for root_dir, sub_dir, files in os.walk(daily_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root_dir, file)
            df = pd.read_csv(file_path)
            df.rename(columns = {'Symbol' : 'Code'}, inplace = True)
            df['Month'] = df['TradingDate'].astype(str).str[:7]
            market = pd.concat([market, df], ignore_index=True)

In [87]:
# 合并dataframe，并计算EP ratio
df_merged = pd.merge(market, returns, on=['TradingDate', 'Code'])
df_merged['EP'] = df_merged['PE'].apply(lambda x: 1/x)
df_merged = df_merged.drop(['PE'], axis=1)
df_merged.head()

,TradingDate,Code,ShortName,CirculatedMarketValue,Month,Dretwd,EP
0,2017-01-03,600000,浦发银行,3.344567e+11,2017-01,0.005552,0.150688
1,2017-01-04,600000,浦发银行,3.350723e+11,2017-01,0.001840,0.150411
2,2017-01-05,600000,浦发银行,3.344567e+11,2017-01,-0.001837,0.150688
3,2017-01-06,600000,浦发银行,3.319945e+11,2017-01,-0.007362,0.151805
4,2017-01-09,600000,浦发银行,3.324049e+11,2017-01,0.001236,0.151618


In [88]:
# 获取每个月月末日期
market['TradingDate'] = pd.to_datetime(market['TradingDate'])
df_dates = market.groupby('Month')['TradingDate'].last()
dates_list = df_dates.tolist()

In [89]:
# 筛选出所有月需要剔除掉的股票代码，并保留前70%的股票代码
result_symbols = []

for date in dates_list:
    df_date = market[market['TradingDate'] == date]
    df_date = df_date.sort_values(by='CirculatedMarketValue', ascending=False)
    num_off = int(len(df_date) * 0.3)   # 计算后30%的股票数量
    symbols = df_date.iloc[-num_off:]['Code'].tolist()   # 得到后30%股票代码
    filtered_codes = df_date.loc[~df_date['Code'].isin(symbols), 'Code'].tolist()  # 保留剔除掉后30%的股票代码

    result_symbols.append(filtered_codes)

In [92]:
# 计算MKT因子

# 读取文件
df_rf = pd.read_csv('TRD_Nrrate d.csv')
df_market_monthly = pd.read_csv('TRD_Mont d.csv')

df_rf['Nrrmtdt'] = df_rf['Nrrmtdt'] / 100  # 把百分比转换为小数

# 按月份分组，并选择每个组的第一个日期对应的Nrrmtdt，存储月度无风险收益到rf_factor中
df_rf['Clsdt'] = pd.to_datetime(df_rf['Clsdt'])
rf_factor = df_rf.groupby(df_rf['Clsdt'].dt.to_period("M"))['Nrrmtdt'].first().tolist()

# 存储市场收益到rm_factor中
df_market_monthly = df_market_monthly[df_market_monthly['Markettype'] == 1]  # 上证A股市场
rm_factor = df_market_monthly['Mretwdos'].tolist()

# 计算MKT因子，rm - rf
MKT_factors_list = np.subtract(rm_factor, rf_factor)

In [93]:
# 计算SMB和HML因子和Rt
size_factors_list = []
value_factors_list = []
Rt_list = []
prvs_SMB = 1
prvs_HML = 1

months_list = df_merged['Month'].unique().tolist()

for index, month in enumerate(months_list):
    symbols_for_month = result_symbols[index]  # 获得当月的股票代码
    df_month = df_merged[(df_merged['Month'] == month) & df_merged['Code'].isin(symbols_for_month)]  # 获得当月市值前70%的股票数据

    # 求Rt
    monthly_return_list_1 = []
    df_daily_stock_rt = df_month.groupby('Code')

    for _, df in df_daily_stock_rt:
        indiv_return = (1 + df['Dretwd']).prod() - 1     # 对于每只股票，计算月收益率
        monthly_return_list_1.append(indiv_return)

    Rt = np.mean(monthly_return_list_1)
    Rt_list.append(Rt)
    
    # 求SMB
    monthly_return_list = []
    df_month_mv = df_month.sort_values(by='CirculatedMarketValue', ascending=False)  # 按照CirculatedMarketValue排序
    df_daily_stock_mv = df_month_mv.groupby('Code')

    for _, df in df_daily_stock_mv:
        indiv_return = (1 + df['Dretwd']).prod() - 1     # 对于每只股票，计算月收益率
        monthly_return_list.append(indiv_return)

    num_bs = int(len(symbols_for_month) * 0.5)
    small_mean = np.mean(monthly_return_list[-num_bs:])  # 市值后50%的股票月收益率
    big_mean = np.mean(monthly_return_list[:num_bs])     # 市值前50%的股票月收益率

    if prvs_SMB < 0:                                     # 若上月SMB<0, 则本月改变策略，用big - small作为size factor
        size_factor = big_mean - small_mean
    else:
        size_factor = small_mean - big_mean                  
    size_factors_list.append(size_factor)

    # 求HML
    monthly_return_list_2 = []
    df_month_ep = df_month.sort_values(by='EP', ascending=False)  # 按照EP ratio排序
    df_daily_stock_ep = df_month_ep.groupby('Code')

    for _, df in df_daily_stock_ep:
        indiv_return_2 = (1 + df['Dretwd']).prod() - 1   # 对于每只股票，计算月收益率
        monthly_return_list_2.append(indiv_return_2)
    
    num_hl = int(len(symbols_for_month) * 0.3)
    high_mean = np.mean(monthly_return_list_2[:num_hl], dtype=np.float64)  # EP前30%的股票月收益率
    low_mean = np.mean(monthly_return_list_2[-num_hl:], dtype=np.float64)  # EP后30%的股票月收益率

    if prvs_HML < 0:                                     # 若上月HML<0, 则本月改变策略，用low - high作为value factor
        value_factor = low_mean - high_mean
    else:
        value_factor = high_mean - low_mean
    value_factors_list.append(value_factor)

    prvs_SMB = size_factor
    prvs_HML = value_factor

In [94]:
# 构建Market cap anomaly和EP anomaly
mc_anomaly = size_factors_list
ep_anomaly = value_factors_list

In [95]:
# 构建CAPM模型: Rit = αi + βi*(Rmt-Rft)
X = np.array(rm_factor).reshape(-1, 1)

y_mc = mc_anomaly
X_with_intercept_mc = sm.add_constant(X)
nw_mc_td = sm.OLS(y_mc, X_with_intercept_mc).fit(cov_type='HAC', cov_kwds={'maxlags': 2})

print('Market cap anomaly:')
print("α:", nw_mc_td.params[0])
print("t(α):", nw_mc_td.tvalues[0])
print("β:", nw_mc_td.params[1])
print("t(β):", nw_mc_td.tvalues[1])

y_ep = ep_anomaly
X_with_intercept_ep = sm.add_constant(X)
nw_ep_td = sm.OLS(y_ep, X_with_intercept_ep).fit(cov_type='HAC', cov_kwds={'maxlags': 2})

print('\n')
print('EP anomaly:')
print("α:", nw_ep_td.params[0])
print("t(α):", nw_ep_td.tvalues[0])
print("β:", nw_ep_td.params[1])
print("t(β):", nw_ep_td.tvalues[1])

Market cap anomaly:
α: 0.0023189456074500974
t(α): 0.6825131950407037
β: -0.0893535862137147
t(β): -1.3987644387812965


EP anomaly:
α: -0.0025789224830507253
t(α): -0.6449106689766356
β: 0.007464516817690132
t(β): 0.06955342003229148


In [96]:
# 构建CH-3模型

X_MKT = np.array(MKT_factors_list).reshape(-1, 1)
X_SMB = np.array(size_factors_list).reshape(-1, 1)
X_VMG = np.array(value_factors_list).reshape(-1, 1)
X = np.concatenate((X_MKT, X_SMB, X_VMG), axis=1)

y_mc = mc_anomaly
X_with_intercept_mc = sm.add_constant(X)
nw_mc_td = sm.OLS(y_mc, X_with_intercept_mc).fit(cov_type='HAC', cov_kwds={'maxlags': 2})

print('Market cap anomaly:')
print("α:", nw_mc_td.params[0])
print("t(α):", nw_mc_td.tvalues[0])
print("βMKT:", nw_mc_td.params[1])
print("t(βMKT):", nw_mc_td.tvalues[1])
print("βSMB:", nw_mc_td.params[2])
print("t(βSMB):", nw_mc_td.tvalues[2])
print("βHML:", nw_mc_td.params[3])
print("t(βHML):", nw_mc_td.tvalues[3])

y_ep = ep_anomaly
X_with_intercept_ep = sm.add_constant(X)
nw_ep_td = sm.OLS(y_ep, X_with_intercept_ep).fit(cov_type='HAC', cov_kwds={'maxlags': 2})

print('\n')
print('EP anomaly:')
print("α:", nw_ep_td.params[0])
print("t(α):", nw_ep_td.tvalues[0])
print("βMKT:", nw_ep_td.params[1])
print("t(βMKT):", nw_ep_td.tvalues[1])
print("βSMB:", nw_ep_td.params[2])
print("t(βSMB):", nw_ep_td.tvalues[2])
print("βHML:", nw_ep_td.params[3])
print("t(βHML):", nw_ep_td.tvalues[3])

Market cap anomaly:
α: -7.059240345053119e-16
t(α): -5.114936731027299
βMKT: -5.898059818321144e-17
t(βMKT): -0.03309396066697366
βSMB: 1.0000000000000002
t(βSMB): 285517552236639.3
βHML: 1.3530843112619095e-16
t(βHML): 0.07551178410509135


EP anomaly:
α: -1.0544950329594016e-15
t(α): -5.116176074615844
βMKT: -2.1510571102112408e-16
t(βMKT): -0.07994418060950899
βSMB: -1.3877787807814457e-17
t(βSMB): -0.0026247701252703395
βHML: 1.0000000000000009
t(βHML): 373583498994714.3
